# GSoC 2023 ML4SCI QML-HEP Tasks

Marçal Comajoan Cara

## Task II: Classical Graph Neural Network (GNN)

### Task statement

For task II, you will use ParticleNet's data for Quark/Gluon jet classification available at [1] with its corresponding description.
- Choose 2 Graph-based architectures of your choice to classify jets as being quarks or gluons. Provide a description on what considerations you have taken to project this point-cloud dataset to a set of interconnected nodes and edges.
- Discuss the resulting performance of the 2 chosen architectures. 

---

### Introduction

The two graph-based architectures I decided to implement are Graph Convolutional Network (GCN) and ParticleNet.

Firstly, I decided to implement the GCN, as introduced in the seminal paper [2][3], because it is one of the most popular graph neural network architectures and serves as the foundation for numerous other graph-based architectures. Secondly, I researched about what are the specific graph neural networks architectures for classifying jets, and I found a recent presentation about jet flavor identification at in the CMS Experiment at CERN [4], which compared several architectures and stated that ParticleNet [5][6] was the one obtaining the best results.

I will implement both architectures using PyTorch [7], as it is my preferred deep learning library. I have opted not to utilize libraries such as PyTorch Geometric, which offer methods for training GNNs more easily, for example by including pre-built graph convolutional layers. My intention with this is to demonstrate how the architectures can be implemented from scratch.

I also use the NumPy [8], SciPy 2-D sparse array  [9] and scikit-learn [10] libraries to perform some operations to process the data.

In [1]:
import os
import math

import numpy as np
import scipy.sparse
import sklearn.model_selection
import sklearn.neighbors
import torch
import torch.nn as nn
import torch.nn.functional as F

def print_model_info(model):
    print(model)
    print("Number of trainable parameters:",
      sum(p.numel() for p in model.parameters() if p.requires_grad))

datasets_dir = os.path.expanduser("~/datasets")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


### Data set

The provided link [1] provides two data sets, each split in 20 files. One dataset contains charm and bottom jets and the other does not. Since here we just want to show how to apply GNNs to these data, we will only use the first file, which does not include the charm and bottom jets, and already provides many samples.

The files are in .npz format, which can be loaded with NumPy.

In [2]:
data = np.load(f'{datasets_dir}/QG_jets/QG_jets.npz')
data.files

['X', 'y']

In [3]:
inputs, labels = data['X'], data['y']

In [4]:
inputs.shape, labels.shape

((100000, 139, 4), (100000,))

The dataset contains 100k jets, 50k are quark jets and 50k gluon jets, randomly sorted.

139 is the max multiplicity of the jets in the file (other jets have been padded with zero-particles), and the features of each particle are its pt, rapidity, azimuthal angle, and pdgid.

The labels are an 0 or 1 to indicate if they are gluon or quark jets, respectively.

Now we perform the train-test split. We use 80% of the data for training and 20% for test.

In [5]:
features_train, features_test, labels_train, labels_test = sklearn.model_selection.train_test_split(inputs, labels, test_size=0.2)

### Graph Convolutional Network

The data as we have it right now is not organized as a graph, which is what we need to apply a GNN. To project the dataset to a set of interconnected nodes and edges, for each jet we construct a graph where each point is a node, and it is connected by an undirected edge to its nearest neighbors. We use the spatial coordinates of the particles in the pseudorapidity-azimuth space, that is, the rapidity and azimuthal angle, which are the second and third features.

We use the nearest neighbors algorithm from scikit-learn, which has a method to get the graph we want. The graph is returned in adjacency matrix format, using a SciPy compressed sparse row (csr) matrix, which is a data structure to store sparse arrays so that it occupies much less memory than storing the full $139\times 139$ array.

Then, we normalize the adjacency matrices using the procedure motivated in [2]:

$$\hat{A} = \tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}$$

where $\tilde{A}=A+I_N$ is the adjacency matrix of the graph with added self-connection, $I_N$ the identity matrix, and $\tilde{D}=\sum_j\tilde{A}_{ij}$.

Finally, we convert the SciPy csr matrices to PyTorch csr tensors, since this is the format we have to use to work with PyTorch.

In [6]:
def create_graphs(features):
    graphs = []
    for x in features[:,:,1:3]:  # rapidity & azimuthal angle
        # kneighbors_graph returns a SciPy csr matrix
        adj_mat = sklearn.neighbors.NearestNeighbors().fit(x).kneighbors_graph(x).sorted_indices()

        # normalize the adjacency matrix
        adj_mat += scipy.sparse.eye(adj_mat.shape[0])
        degree_matrix = scipy.sparse.diags(np.asarray(np.power(adj_mat.sum(axis=1), 0.5)).flatten())
        adj_mat = degree_matrix @ adj_mat @ degree_matrix
        
        # now convert the SciPy csr matrices to PyTorch csr tensors
        adj_mat = torch.sparse_csr_tensor(torch.tensor(adj_mat.indptr), torch.tensor(adj_mat.indices), torch.tensor(adj_mat.data),
                                          size=torch.Size(adj_mat.shape), dtype=torch.float32)
        
        graphs.append(adj_mat)
        
    return graphs

We also convert the features and labels NumPy arrays to PyTorch tensors.

In [7]:
def create_dataset(features, labels):
    graphs = create_graphs(features)
    features = torch.tensor(features, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.float32)
    return graphs, features, labels

In [8]:
train_set = create_dataset(features_train, labels_train)
test_set = create_dataset(features_test, labels_test)

(len(train_set[0]), train_set[0][0].shape), train_set[1].shape, train_set[2].shape

/tmp/ipykernel_391697/1367554803.py:13: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /opt/conda/conda-bld/pytorch_1678402412426/work/aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  adj_mat = torch.sparse_csr_tensor(torch.tensor(adj_mat.indptr), torch.tensor(adj_mat.indices), torch.tensor(adj_mat.data),


((80000, torch.Size([139, 139])),
 torch.Size([80000, 139, 4]),
 torch.Size([80000]))

We see that for training we have 80000 samples. Each input to the model is composed of the $139\times 139$ normalized adjacency matrix, and $139\times 4$ features.

In [9]:
(train_set[2] == 0).sum().item()

39936

We observe that the data set is balanced, as almost half of the dataset is from one class, and the other half from the other.

Now let us define the graph convolution layer and the graph convolutional network!

The graph convolutional network has two graph convolution layers with a dropout in between, and is followed by a readout layer and a final fully connected layer.

In [10]:
class GraphConv(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.weight = nn.Parameter(torch.Tensor(in_dim, out_dim))
        self.bias = nn.Parameter(torch.Tensor(out_dim))
        self.reset_parameters()
        
    def reset_parameters(self):
        stdv = 1 / math.sqrt(self.weight.size(1))
        nn.init.uniform_(self.weight, -stdv, stdv)
        nn.init.uniform_(self.bias, -stdv, stdv)
    
    def forward(self, node_feats, adj_mat):
        support = torch.mm(node_feats, self.weight)
        output = torch.mm(adj_mat, support) + self.bias
        return output
    
class GCN(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim, dropout):
        super().__init__()
        self.gc1 = GraphConv(in_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)
        self.gc2 = GraphConv(hid_dim, hid_dim)
        self.lin = nn.Linear(hid_dim, out_dim)
    
    def forward(self, node_feats, adj_mat):
        x = self.gc1(node_feats, adj_mat)
        x = x.relu()
        
        x = self.dropout(x)
        
        x = self.gc2(x, adj_mat)
        x = x.relu()
        
        x = torch.mean(x, dim=0) # readout layer
        
        x = self.lin(x)
        return x

gcn = GCN(inputs.shape[-1], 128, 1, 0.1).to(device)
print_model_info(gcn)

GCN(
  (gc1): GraphConv()
  (dropout): Dropout(p=0.1, inplace=False)
  (gc2): GraphConv()
  (lin): Linear(in_features=128, out_features=1, bias=True)
)
Number of trainable parameters: 17281


Using 128 as the hidden layer dimension we obtain that we will need to train 17281 parameters.

Now let us train the network. Since the model we defined does not support batched input, we will use a batch size of 1.

In [11]:
def train(model, device, train_set, optimizer):
    correct = 0
    model.train()
    criterion = nn.BCEWithLogitsLoss()
    for adj_mat, node_feats, label in zip(*train_set):
        adj_mat, node_feats, label = adj_mat.to(device), node_feats.to(device), label.to(device)
        optimizer.zero_grad()
        out = model(node_feats, adj_mat)
        loss = criterion(out, label.unsqueeze(0))
        loss.backward()
        optimizer.step()
        correct += ((F.sigmoid(out) > 0.5) == label).item()
    return correct / len(train_set[1])
    
def test(model, device, test_set):
    correct = 0
    model.eval()
    with torch.no_grad():
        for adj_mat, node_feats, label in zip(*test_set):
            adj_mat, node_feats, label = adj_mat.to(device), node_feats.to(device), label.to(device)
            out = model(node_feats, adj_mat)
            correct += ((F.sigmoid(out) > 0.5) == label).item()
    return correct / len(test_set[1])

optimizer = torch.optim.Adam(gcn.parameters())

for epoch in range(5):
    print(f"Epoch {epoch}")
    print(f' TRAINING\tAccuracy: {train(gcn, device, train_set, optimizer):.6f}')
    print(f" TESTING \tAccuracy: {test(gcn, device, test_set):.6f}")

Epoch 0
 TRAINING	Accuracy: 0.660875
 TESTING 	Accuracy: 0.661200
Epoch 1
 TRAINING	Accuracy: 0.664613
 TESTING 	Accuracy: 0.666950
Epoch 2
 TRAINING	Accuracy: 0.663963
 TESTING 	Accuracy: 0.666800
Epoch 3
 TRAINING	Accuracy: 0.663037
 TESTING 	Accuracy: 0.663050
Epoch 4
 TRAINING	Accuracy: 0.662388
 TESTING 	Accuracy: 0.661350


We observe that after the first epoch, the model does not improve much.

The final test accuracy obtained is 66.14%. Let us see now if ParticleNet performs better.

### ParticleNet

ParticleNet also constructs the graph from the point cloud data with the approach we used for the GCN. However, in the original implementation this step is already included inside the network definition, in particular, it is performed inside the layer called `EdgeConv`.

Nonetheless, we still have to separate the coordinates in a separated tensor from the features, which is what we do here, apart from converting from NumPy arrays to PyTorch tensors as before. We also permute the dimensions to have the order expected in the ParticleNet implementation, and create PyTorch Datasets and DataLoaders to train in batches, in particular of 32 samples each.

In [12]:
def create_dataset(features, labels):
    points = torch.tensor(features[:,:,1:3], dtype=torch.float32).permute(0, 2, 1)
    features = torch.tensor(features, dtype=torch.float32).permute(0, 2, 1)
    labels = torch.tensor(labels, dtype=torch.float32)
    return torch.utils.data.TensorDataset(points, features, labels)

train_set = create_dataset(features_train, labels_train)
test_set = create_dataset(features_test, labels_test)

train_dataloader =  torch.utils.data.DataLoader(train_set, batch_size=32)
test_dataloader =  torch.utils.data.DataLoader(test_set, batch_size=32)

Now let us define the `EdgeConv` layer and ParticleNet!

The architecture implemented is the same one as the original, except for the hyperparameter values, which are set so that the ParticleNet has a similar number of parameters to the GCN, to make the comparison more fair. In particular, we are using two `EdgeConv` layers.

In [13]:
class EdgeConv(nn.Module):
    def __init__(self, k, in_feat, out_feats):
        super().__init__()
        
        self.k = k
        
        self.convs = nn.ModuleList([nn.Conv2d(2 * in_feat if i == 0 else out_feats[i-1],
                                              out_feats[i],
                                              kernel_size=1,
                                              bias=False)
                                    for i in range(len(out_feats))])
        self.bns = nn.ModuleList([nn.BatchNorm2d(out_feats[i])
                                  for i in range(len(out_feats))])
        self.acts = nn.ModuleList([nn.ReLU()
                                   for i in range(len(out_feats))])
        if in_feat == out_feats[-1]:
            self.sc = None
        else:
            self.sc = nn.Conv1d(in_feat, out_feats[-1], kernel_size=1, bias=False)
            self.sc_bn = nn.BatchNorm1d(out_feats[-1])
        self.sc_act = nn.ReLU()

    def forward(self, points, features):
        # KNN
        x = features
        inner = -2 * torch.matmul(x.transpose(2, 1), x)
        xx = torch.sum(x**2, dim=1, keepdim=True)
        pairwise_distance = -xx - inner - xx.transpose(2, 1)
        topk_indices = pairwise_distance.topk(k=self.k + 1, dim=-1)[1][:,:,1:]
        
        # Get graph features
        x = features
        batch_size, num_dims, num_points = features.shape
        idx_base = torch.arange(0, batch_size, device=features.device).view(-1, 1, 1) * num_points
        idx = topk_indices + idx_base
        idx = idx.view(-1)
        fts = x.transpose(2, 1).reshape(-1, num_dims)
        fts = fts[idx,:].view(batch_size, num_points, self.k, num_dims)
        fts = fts.permute(0, 3, 1, 2).contiguous()
        x = x.view(batch_size, num_dims, num_points, 1).repeat(1, 1, 1, self.k)
        x = torch.cat((x, fts - x), dim=1)
        
        # Apply EdgeConv
        for conv, bn, act in zip(self.convs, self.bns, self.acts):
            x = conv(x)
            x = bn(x)
            x = act(x)
        
        fts = x.mean(-1)
        
        if self.sc:
            sc = self.sc(features)
            sc = self.sc_bn(sc)
        else:
            sc = features
        
        return self.sc_act(sc + fts)
    
class ParticleNet(nn.Module):
    def __init__(self, in_dims, out_dim, conv_params, fc_params):
        super().__init__()
        
        self.bn_fts = nn.BatchNorm1d(in_dims)
        
        self.edge_convs = nn.ModuleList([EdgeConv(conv_params[i][0],
                                                  in_dims if i == 0 else conv_params[i-1][1][-1],
                                                  conv_params[i][1])
                                         for i in range(len(conv_params))])
        
        in_chn = sum(x[-1] for _, x in conv_params)
        out_chn = np.clip((in_chn // 128) * 128, 128, 1024)
        self.fusion_block = nn.Sequential(
            nn.Conv1d(in_chn, out_chn, kernel_size=1, bias=False),
            nn.BatchNorm1d(out_chn),
            nn.ReLU()
        )

        self.fc = nn.ModuleList([nn.Sequential(nn.Linear(out_chn if i == 0 else fc_params[i-1][0],
                                                         fc_params[i][0]),
                                               nn.ReLU(),
                                               nn.Dropout(fc_params[i][1]))
                                 for i in range(len(fc_params))])
        self.fc.append(nn.Linear(fc_params[-1][0], out_dim))
    
    def forward(self, points, features):
        mask = (features.abs().sum(dim=1, keepdim=True) != 0)
        points *= mask
        features *= mask
        coord_shift = (mask == 0) * 1e9
        
        counts = mask.float().sum(dim=-1)
        counts = torch.max(counts, torch.ones_like(counts))
        
        fts = self.bn_fts(features) * mask
        
        outputs = []
        for i, conv in enumerate(self.edge_convs):
            pts = (points if i == 0 else fts) + coord_shift
            fts = conv(pts, fts) * mask
            outputs.append(fts)
        fts = self.fusion_block(torch.cat(outputs, dim=1)) * mask
        
        x = fts.sum(dim=-1) / counts
        
        for lyr in self.fc:
            x = lyr(x)
        return x

particle_net = ParticleNet(4, 1, conv_params=[(5, (8, 8, 8)), (5, (16, 16, 16))], fc_params=[(128, 0.1)]).to(device)
print_model_info(particle_net)

ParticleNet(
  (bn_fts): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (edge_convs): ModuleList(
    (0): EdgeConv(
      (convs): ModuleList(
        (0-2): 3 x Conv2d(8, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (bns): ModuleList(
        (0-2): 3 x BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (acts): ModuleList(
        (0-2): 3 x ReLU()
      )
      (sc): Conv1d(4, 8, kernel_size=(1,), stride=(1,), bias=False)
      (sc_bn): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (sc_act): ReLU()
    )
    (1): EdgeConv(
      (convs): ModuleList(
        (0-2): 3 x Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (bns): ModuleList(
        (0-2): 3 x BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (acts): ModuleList(
        (0-2): 3 x ReLU()
      )
      (sc): Conv1d(8, 16, 

We will have to train 21289 parameters. Recall that the GCN trained had 17281.

Now let us train the network.

In [14]:
def train(model, device, train_dataloader, optimizer):
    correct = 0
    model.train()
    criterion = nn.BCEWithLogitsLoss()
    for points, features, label in train_dataloader:
        points, features, label = points.to(device), features.to(device), label.to(device)
        optimizer.zero_grad()
        out = model(points, features).squeeze()
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        correct += ((F.sigmoid(out) > 0.5) == label).sum().item()
    return correct / len(train_dataloader.dataset)
    
def test(model, device, test_dataloader):
    correct = 0
    model.eval()
    with torch.no_grad():
        for points, features, label in test_dataloader:
            points, features, label = points.to(device), features.to(device), label.to(device)
            out = model(points, features).squeeze()
            correct += ((F.sigmoid(out) > 0.5) == label).sum().item()
    return correct / len(test_dataloader.dataset)

optimizer = torch.optim.Adam(particle_net.parameters())

for epoch in range(5):
    print(f"Epoch {epoch}")
    print(f' TRAINING\tAccuracy: {train(particle_net, device, train_dataloader, optimizer):.6f}')
    print(f" TESTING \tAccuracy: {test(particle_net, device, test_dataloader):.6f}")

Epoch 0
 TRAINING	Accuracy: 0.768575
 TESTING 	Accuracy: 0.781950
Epoch 1
 TRAINING	Accuracy: 0.776975
 TESTING 	Accuracy: 0.783450
Epoch 2
 TRAINING	Accuracy: 0.778737
 TESTING 	Accuracy: 0.784450
Epoch 3
 TRAINING	Accuracy: 0.780637
 TESTING 	Accuracy: 0.784100
Epoch 4
 TRAINING	Accuracy: 0.781450
 TESTING 	Accuracy: 0.784600


With ParticleNet we obtained an accuracy of 78.46% on the test set.

### Conclusion

The GCN obtained an accuracy of 66.14% while ParticleNet achieved an accuracy of 78.46%, which is an increase of 12.32%.

Possibly, this increase is due to the architecture, which is more tailored for the task at hand. However, we also have to consider that the number of parameters in ParticleNet was slightly higher than in the GCN model.

Perhaps we could achieve better accuracies in both models by using more data from the files we did not use and by performing hyperparameter optimization, but this is outside the scope of this task.

### References

1. [Pythia8 Quark and Gluon Jets for Energy Flow data set (Zenodo)](https://zenodo.org/record/3164691#.YigdGt9MHrB)
2. [Thomas N. Kipf, Max Welling. Semi-Supervised Classification with Graph Convolutional Networks (2017)](https://arxiv.org/abs/1609.02907)
3. [Graph Convolutional Networks in PyTorch (Thomas Kipf GitHub)](https://github.com/tkipf/pygcn)
4. [Loukas Gouskos. Jet flavor identification using Graph Neural Networks in CMS (2022)](https://indico.cern.ch/event/1201307/attachments/2522301/4337334/lg-cernds-gnntaggingcms_v2.pdf)
5. [Huilin Qu, Loukas Gouskos. ParticleNet: Jet Tagging via Particle Clouds (2020)](https://arxiv.org/abs/1902.08570)
6. [ParticleNet (CMS Machine Learning Documentation)](https://cms-ml.github.io/documentation/inference/particlenet.html)
7. [PyTorch](https://pytorch.org/)
8. [NumPy](https://numpy.org/)
9. [SciPy sparse matrices documentation](https://docs.scipy.org/doc/scipy/reference/sparse.html)
10. [scikit-lern](https://scikit-learn.org/stable/index.html)
